In [1]:
import pandas
import numpy as np
import sys
import zipfile, os, time
import urllib3
from datetime import datetime, timedelta
import glob
import h5py
import plotly.express as px
fs = open('path.txt', 'r')
path = fs.read()  
fs.close()
sys.path.append(path)

import cds_eua3 as eua
eua.logging_set_level(30)
import xarray as xr

import cdsapi, zipfile, os, time

In [2]:
import numpy
import pandas as pd
def write_dict_h5(dfile, f, k, fbencodings, var_selection=[], mode='a', attrs={}): 
    """ Writes each separate variable from the observation or feedback tables inot netcdf using h5py.
          f is a pandas dataframe with one column, one for each variable
          k is either 'era5fb' or 'observations_table'
          fbencodings is the encodings of variable types, e.g. {'observations_id': { 'compression': 'gzip' } }
    """

    #attrs=  {'date_time':('units','seconds since 1900-01-01 00:00:00')}
    #attrs = {'observation_id': ('description', 'unique ID for observation'), 'report_id': ('description', 'Link to header information') , 'date_time':('units','seconds since 1900-01-01 00:00:00') }
    
    with h5py.File(dfile,mode) as fd:
        if k == '':
            v = var_selection[0]
            fd.create_dataset(v,f[v].shape,f[v].dtype,compression=fbencodings[v]['compression'], chunks=True)
            fd[v][:]=f[v].values[:]
        else:
            try:
                fd.create_group(k)
                index=numpy.zeros (f[list(f.keys())[0]].shape[0], dtype='S1')
                fd[k].create_dataset('index', data=index)
            except:
                pass
            if not var_selection:
                var_selection=list(f.keys())
            fixed_string_len = 20
            string10=numpy.zeros(fixed_string_len,dtype='S1')
            sdict={}
            slist=[]

            #groupencodings     

            for v in var_selection:          
                #variables_dic[v] = ''
                if type(f[v]) == pd.core.series.Series:
                    fvv=f[v].values
                else:
                    fvv=f[v]

                if type(fvv[0]) not in [str,bytes,numpy.bytes_]:

                    if fvv.dtype !='S1':

                        fd[k].create_dataset(v,fvv.shape,fvv.dtype,compression=fbencodings[v]['compression'], chunks=True)
                        fd[k][v][:]=fvv[:]
                        if attrs:    #  attrs={'date_time':('units','seconds since 1900-01-01 00:00:00')}
                            if v in attrs.keys():
                                for kk,vv in attrs[v].items():
                                    if type(vv) is str:  
                                        fd[k][v].attrs[kk]=numpy.bytes_(vv)
                                    else:
                                        fd[k][v].attrs[kk]=vv

                        if v in ['date_time','report_timestamp','record_timestamp']:
                            fd[k][v].attrs['units']=numpy.bytes_('seconds since 1900-01-01 00:00:00')                            #print (  fk, ' ' , v , ' ' ,   ) 

                    else:
                        fd[k].create_dataset(v,fvv.shape,fvv.dtype,compression=fbencodings[v]['compression'], chunks=True)
                        fd[k][v][:]=fvv[:]
                        slen=fvv.shape[1]
                        sdict[v]=slen
                        if slen not in slist:
                            slist.append(slen)
                            try:
                                fd[k].create_dataset( 'string{}'.format(slen),  data=string10[:slen]  )
                            except:
                                pass               
                        if v in attrs.keys():
                            fd[k][v].attrs['description']=numpy.bytes_(attrs[v]['description'])
                            fd[k][v].attrs['external_table']=numpy.bytes_(attrs[v]['external_table'])

                else:
                    sleno=len(fvv[0])
                    slen=sleno
                    try:
                        slen=int(fvv.dtype.descr[0][1].split('S')[1])
                    except:  
                        pass

                    sdict[v]=slen
                    if slen not in slist:
                        slist.append(slen)
                        try:
                            print('trying to cread dim')
                            fd[k].create_dataset( 'string{}'.format(slen),  data=string10[:slen]  )
                            print('dim created')
                        except:
                            print('dim creation failed')
                            pass               

                    #x=x.reshape(fvv.shape[0],slen)
                    fd[k].create_dataset(v,data=fvv.view('S1').reshape(fvv.shape[0],slen),compression=fbencodings[v]['compression'],chunks=True)
                    if v in attrs.keys():
                        fd[k][v].attrs['description']     =numpy.bytes_(attrs[v]['description'])
                        fd[k][v].attrs['external_table']=numpy.bytes_(attrs[v]['external_table'])                

                #variables_dic[v] = f[v].values.dtype

            for v in fd[k].keys(): #var_selection:
                l=0      

                '''
                if v == 'primary_station_id':
                    try:
                        fd[k][v].dims[l].attach_scale(fd[k]['index'])
                    except:
                        pass

                    try:
                        slen =  len( fd[k][v][0] )
                        stringa=numpy.zeros( slen , dtype='S1')
                        fd[k].create_dataset( 'string{}'.format(slen),  data= stringa  )                                        
                        fd[k][v].dims[1].attach_scale(   fd[k]['string{}'.format(slen)]  )                                        
                    except:
                        fd[k][v].dims[1].attach_scale(   fd[k]['string{}'.format(slen)]  )                    


                if v == 'station_name':
                    try:
                        fd[k][v].dims[l].attach_scale(fd[k]['index'])
                        slen =  len( fd[k][v][0][0])
                        stringa=numpy.zeros( slen , dtype='S1')
                    except:
                        pass
                    try:
                        fd[k].create_dataset( 'string{}'.format(slen),  data= stringa  )
                        fd[k][v].dims[1].attach_scale(   fd[k]['string{}'.format(slen)]  )                    
                        print('done attaching')
                    except:
                        print('not working')

                '''             
                try:
                    if type(f[v]) == pd.core.series.Series:
                        fvv=f[v].values
                    else:
                        fvv=f[v]
                    if 'string' not in v and v!='index':                    
                        fd[k][v].dims[l].attach_scale(fd[k]['index'])
                        #print(v,fvv.ndim,type(fvv[0]))
                        if fvv.ndim==2 or type(fvv[0]) in [str,bytes,numpy.bytes_]:
                            slen=sdict[v]
                            #slen=10
                            fd[k][v].dims[1].attach_scale(fd[k]['string{}'.format(slen)])
                except:
                    pass



            i=4        
            for v in slist:
                s='string{}'.format(v)
                for a in ['NAME']:
                    fd[k][s].attrs[a]=numpy.bytes_('This is a netCDF dimension but not a netCDF variable.')

                i+=1

    return
#variables_dic = {}

In [3]:
# files = glob.glob('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11035*.nc')
files = glob.glob('/raid60/scratch/uli/0*.nc')
files[0]

'/raid60/scratch/uli/0-20000-0-93062_CEUAS_merged_v0.nc'

In [4]:
data = eua.CDMDataset(files[0])
data

File: <HDF5 file "0-20000-0-93062_CEUAS_merged_v0.nc" (mode r)>
Filename: /raid60/scratch/uli/0-20000-0-93062_CEUAS_merged_v0.nc
(G)roups/(V)ariables: 

 - G | crs__________________________________________ : : 4
 - V | dateindex____________________________________ : : (112,)
 - G | era5fb_______________________________________ : : 72
 - G | header_table_________________________________ : : 56
 - G | observations_table___________________________ : : 50
 - G | observed_variable____________________________ : : 9
 - V | recordindex__________________________________ : : (167,)
 - V | recordtimestamp______________________________ : : (167,)
 - G | sensor_configuration_________________________ : : 12
 - G | source_configuration_________________________ : : 2
 - G | station_configuration________________________ : : 47
 - G | station_configuration_codes__________________ : : 7
 - G | station_type_________________________________ : : 4
 - G | units________________________________________ : : 6
 

In [5]:
allvars = data.observations_table.observed_variable[()]
np.unique(allvars)

array([  0,  36,  38,  39,  85, 104, 105, 106, 107, 117, 136, 137, 138,
       139, 140])

In [6]:
#
#
allvars = data.observations_table.observed_variable[()]
allvars.sort()
allvars = np.unique(allvars)
#
ri = data.recordindex[()]
rt = data.recordtimestamp[()]
keys = data.observations_table.keys()[:-1]
# dropping all keys, where dimensions won't work - just help variabels for dimensions
keys.pop(43)
keys.pop(42)
keys.pop(41)
recordindices = [[] for i in range(len(np.unique(allvars)))]
recordtimestamps = [[] for i in range(len(np.unique(allvars)))]

# output variables (from observations_table)
ov = []
for o in keys:
    ov.append([[] for i in range(len(np.unique(allvars)))])
    
for i in range(len(ri)):
    try:
        start = ri[i]
        end = ri[i+1]
    except:
        break
        
    a = data.observations_table.observed_variable[()][start:end]
    helpvar = []
    for o in range(len(keys)):
        if keys[o] == 'observation_id' or keys[o] == 'report_id' or keys[o] == 'sensor_id' or keys[o] == 'source_id':
            b = []
            for n in data.observations_table[keys[o]][()][start:end]:
                c = ''
                for bb in n:
                    c = c + bb.decode()
                b.append(c)
        else:
            b = data.observations_table[keys[o]][()][start:end]
        sa, sb = zip(*sorted(zip(a, b)))
        helpvar.append(sb)

    for j in range(len(allvars)):
        for k in range(len(sa)):
            if sa[k] == allvars[j]:
                for m in range(len(helpvar)):
                    ov[m][j].append(helpvar[m][k])
        recordindices[j].append(len(ov[0][j]))
        recordtimestamps[j].append(rt[i])
        
#
# setting record_indices to the right value
#
lenadd = 0.
for i in range(len(recordindices)):
    recordindices[i] = np.asarray(np.append([0], recordindices[i][:-1])) + lenadd
    lenadd += len(ov[0][i])

#
# shaping record_indices -> setting every missing value to nan
#
old = -1
for i in range(len(recordindices)):
    for j in range(len(recordindices[i])):
        if recordindices[i][j] != 0 and recordindices[i][j] == recordindices[i][j-1]:
            old = recordindices[i][j]
            recordindices[i][j] = np.nan
        elif recordindices[i][j] == old:
            recordindices[i][j] = np.nan

#
# recordtimestamps are only necessary once
#
recordtimestamps = recordtimestamps[0]

# 
# stacking all output variables
#
out = []
for j in ov:
    finalvar = []
    for i in j:
        finalvar = finalvar + i
    out.append(finalvar)
    
#
# restoring byte arrays:
#
for o in range(len(keys)):
    if keys[o] == 'observation_id' or keys[o] == 'report_id' or keys[o] == 'sensor_id' or keys[o] == 'source_id':
        b = []
        for n in range(len(out[o])):
            c = []
            for bb in out[o][n]:
                c.append(bb.encode())
            out[o][n] = c


In [7]:
#
# writing data into observations_table
#

file = h5py.File('/raid60/scratch/uli/testoutputfile.nc', 'r+')

for i in range(len(keys)):
    try:
        del file['observations_table'][keys[i]]
    except:
        pass
    if keys[i] == 'index':
        pass
    elif keys[i] == 'observation_id' or keys[i] == 'report_id' or keys[i] == 'sensor_id' or keys[i] == 'source_id':
        slen = len(out[i][0])
        alldict = {keys[i]:np.asarray(out[i], dtype='S{}'.format(slen))}
        write_dict_h5('/raid60/scratch/uli/testoutputfile.nc', alldict, 'observations_table', {keys[i]: { 'compression': 'gzip' } }, [keys[i]])
    else:
        alldict = pandas.DataFrame({keys[i]:out[i]})
        write_dict_h5('/raid60/scratch/uli/testoutputfile.nc', alldict, 'observations_table', {keys[i]: { 'compression': 'gzip' } }, [keys[i]])  
#
# writing the recordindices and recordtimestamp.
#       
for i in range(len(recordindices)):
    file = h5py.File('/raid60/scratch/uli/testoutputfile.nc', 'r+')
    try:
        del file['recordindices'][str(allvars[i])]
    except:
        pass
    testvar = pandas.DataFrame({str(allvars[i]):recordindices[i]})
    write_dict_h5('/raid60/scratch/uli/testoutputfile.nc', testvar, 'recordindices', {str(allvars[i]): { 'compression': 'gzip' } }, [str(allvars[i])]) 

try:
    del file['recordindex']
except:
    pass
try:
    del file['recordtimestamp']
except:
    pass

testvar = pandas.DataFrame({'recordtimestamp':recordtimestamps})
write_dict_h5('/raid60/scratch/uli/testoutputfile.nc', testvar, '', {'recordtimestamp': { 'compression': 'gzip' } }, ['recordtimestamp']) 

In [8]:
data_test = eua.CDMDataset('/raid60/scratch/uli/testoutputfile.nc')
data_test

File: <HDF5 file "testoutputfile.nc" (mode r+)>
Filename: /raid60/scratch/uli/testoutputfile.nc
(G)roups/(V)ariables: 

 - G | crs__________________________________________ : : 4
 - V | dateindex____________________________________ : : (112,)
 - G | era5fb_______________________________________ : : 72
 - G | header_table_________________________________ : : 56
 - G | observations_table___________________________ : : 49
 - G | observed_variable____________________________ : : 9
 - G | recordindices________________________________ : : 16
 - V | recordtimestamp______________________________ : : (166,)
 - G | sensor_configuration_________________________ : : 12
 - G | source_configuration_________________________ : : 2
 - G | station_configuration________________________ : : 47
 - G | station_configuration_codes__________________ : : 7
 - G | station_type_________________________________ : : 4
 - G | units________________________________________ : : 6
 - G | z_coordinate_type______________

In [9]:
# Example: temperature
data_test.recordindices['85'][()]

array([  nan, 2393., 2419., 2432., 2457., 2481., 2495., 2512., 2527.,
       2555., 2568., 2578., 2588., 2601., 2619., 2637., 2651., 2668.,
       2685., 2703., 2720., 2732., 2759., 2772., 2784., 2793., 2811.,
       2829., 2843., 2857., 2873., 2890., 2905., 2919., 2931., 2946.,
       2962., 2978., 2995., 3008., 3021., 3038., 3061., 3088., 3105.,
       3122., 3135., 3152., 3165., 3180., 3197., 3211., 3223., 3237.,
       3262., 3272.,   nan, 3300., 3312., 3326., 3339., 3357., 3371.,
       3383., 3396., 3411., 3429., 3441., 3454., 3466., 3484., 3494.,
       3507., 3519., 3532., 3545., 3572., 3585., 3597., 3615., 3628.,
       3641., 3659., 3687., 3700., 3713., 3739., 3752., 3770., 3783.,
       3796., 3809., 3822., 3839., 3852., 3865., 3878., 3890., 3908.,
       3921., 3934., 3947., 3961., 3975., 3988., 4001., 4014., 4032.,
       4045., 4059., 4073., 4086., 4099., 4112., 4130., 4144., 4161.,
       4178., 4191., 4204., 4217., 4230., 4243., 4260., 4274., 4287.,
       4300., 4328.,

In [10]:
data_test.recordindices

recordindices:

0_________________________________________________ : : (166,)
104_______________________________________________ : : (166,)
105_______________________________________________ : : (166,)
106_______________________________________________ : : (166,)
107_______________________________________________ : : (166,)
117_______________________________________________ : : (166,)
136_______________________________________________ : : (166,)
137_______________________________________________ : : (166,)
138_______________________________________________ : : (166,)
139_______________________________________________ : : (166,)
140_______________________________________________ : : (166,)
36________________________________________________ : : (166,)
38________________________________________________ : : (166,)
39________________________________________________ : : (166,)
85________________________________________________ : : (166,)
index_____________________________________________ : :

In [11]:
data_test.observations_table

observations_table:

adjustment_id_____________________________________ : : (30269,)
advanced_assimilation_feedback____________________ : : (30269,)
advanced_homogenisation___________________________ : : (30269,)
advanced_qc_______________________________________ : : (30269,)
advanced_uncertainty______________________________ : : (30269,)
bbox_max_latitude_________________________________ : : (30269,)
bbox_max_longitude________________________________ : : (30269,)
bbox_min_latitude_________________________________ : : (30269,)
bbox_min_longitude________________________________ : : (30269,)
code_table________________________________________ : : (30269,)
conversion_flag___________________________________ : : (30269,)
conversion_method_________________________________ : : (30269,)
crs_______________________________________________ : : (30269,)
data_policy_licence_______________________________ : : (30269,)
date_time_________________________________________ : : (30269,)
date_time_meaning__

In [12]:
data.recordtimestamp[()]

array([1897480800, 1897502400, 1897592400, 1899385200, 1899406800,
       1899493200, 1900098000, 1900360800, 1900368000, 1900447200,
       1900533600, 1900684800, 1900702800, 1901545200, 1901631600,
       1901829600, 1902430800, 1902528000, 1902776400, 1902949200,
       1903024800, 1903449600, 1903464000, 1903640400, 1904076000,
       1904133600, 1904569200, 1904587200, 1904745600, 1904760000,
       1905379200, 1905796800, 1905804000, 1905969600, 1906048800,
       1906059600, 1906070400, 1906495200, 1906567200, 1906578000,
       1906750800, 1907096400, 1907107200, 1907179200, 1907186400,
       1907258400, 1908554400, 1908878400, 1908892800, 1908972000,
       1909000800, 1909051200, 1909065600, 1909159200, 1909173600,
       1909494000, 1909519200, 1909584000, 1909605600, 1910088000,
       1910098800, 1910260800, 1910358000, 1910606400, 1910613600,
       1910620800, 1910725200, 1910865600, 1910952000, 1910962800,
       1910988000, 1911391200, 1911398400, 1911470400, 1911477

In [13]:
data_test.recordtimestamp[()]

array([1897480800, 1897502400, 1897592400, 1899385200, 1899406800,
       1899493200, 1900098000, 1900360800, 1900368000, 1900447200,
       1900533600, 1900684800, 1900702800, 1901545200, 1901631600,
       1901829600, 1902430800, 1902528000, 1902776400, 1902949200,
       1903024800, 1903449600, 1903464000, 1903640400, 1904076000,
       1904133600, 1904569200, 1904587200, 1904745600, 1904760000,
       1905379200, 1905796800, 1905804000, 1905969600, 1906048800,
       1906059600, 1906070400, 1906495200, 1906567200, 1906578000,
       1906750800, 1907096400, 1907107200, 1907179200, 1907186400,
       1907258400, 1908554400, 1908878400, 1908892800, 1908972000,
       1909000800, 1909051200, 1909065600, 1909159200, 1909173600,
       1909494000, 1909519200, 1909584000, 1909605600, 1910088000,
       1910098800, 1910260800, 1910358000, 1910606400, 1910613600,
       1910620800, 1910725200, 1910865600, 1910952000, 1910962800,
       1910988000, 1911391200, 1911398400, 1911470400, 1911477

---

In [14]:
# allvars = data.observations_table.observed_variable[()]
# allvars.sort()
# allvars = np.unique(allvars)
# # recordindices einzeln sortieren - für alle variablen eigene listen erstellen - dazu recordindices erstellen - alle daten hintereinander legen - len aller vorhergehender rec_in zu den nachkommenden addieren.
# #
# ri = data.recordindex[()]
# rt = data.recordtimestamp[()]
# keys = data.observations_table.keys()
# recordindices = [[] for i in range(len(np.unique(allvars)))]
# recordtimestamps = [[] for i in range(len(np.unique(allvars)))]

# variables = [[] for i in range(len(np.unique(allvars)))]
# values = [[] for i in range(len(np.unique(allvars)))]
# zcoords = [[] for i in range(len(np.unique(allvars)))]

# for i in range(len(ri)):
#     try:
#         start = ri[i]
#         end = ri[i+1]
#     except:
#         break
        
#     a = data.observations_table.observed_variable[()][start:end]
#     b = data.observations_table.observation_value[()][start:end]
#     c = data.observations_table.z_coordinate[()][start:end]

#     sa, sb = zip(*sorted(zip(a, b)))
#     sa, sc = zip(*sorted(zip(a, c)))
#     for j in range(len(allvars)):
#         for k in range(len(sa)):
#             if sa[k] == allvars[j]:
#                 variables[j].append(sa[k])
#                 values[j].append(sb[k])
#                 zcoords[j].append(sc[k])
#         recordindices[j].append(len(variables[j]))
#         recordtimestamps[j].append(rt[i])
        
# lenadd = 0
# for i in range(len(recordindices)):
#     recordindices[i] = np.asarray(np.append([0], recordindices[i][:-1])) + lenadd
#     lenadd += len(variables[i])


In [15]:
# allvars = data.observations_table.observed_variable[()]
# allvars.sort()
# allvars = np.unique(allvars)
# #
# ri = data.recordindex[()]
# rt = data.recordtimestamp[()]
# keys = data.observations_table.keys()[:-1]
# recordindices = [[] for i in range(len(np.unique(allvars)))]
# recordtimestamps = [[] for i in range(len(np.unique(allvars)))]

# # output variables (from observations_table)
# ov = []
# for o in keys:
#     ov.append([[] for i in range(len(np.unique(allvars)))])
    
# for i in range(len(ri)):
#     try:
#         start = ri[i]
#         end = ri[i+1]
#     except:
#         break
        
#     a = data.observations_table.observed_variable[()][start:end]
#     helpvar = []
#     for o in range(len(keys)):
#         b = data.observations_table[keys[o]][()][start:end]
#         try: # all varibales with different shape won't work here
#             sa, sb = zip(*sorted(zip(a, b)))
#             helpvar.append(sb)
#         except:
#             pass

#     for j in range(len(allvars)):
#         for k in range(len(sa)):
#             if sa[k] == allvars[j]:
#                 for m in range(len(helpvar)):
#                     try: # all varibales with different shape won't work here
#                         ov[m][j].append(helpvar[m][k])
#                     except:
#                         pass
#         recordindices[j].append(len(ov[0][j]))
#         recordtimestamps[j].append(rt[i])
        
# lenadd = 0
# for i in range(len(recordindices)):
#     recordindices[i] = np.asarray(np.append([0], recordindices[i][:-1])) + lenadd
#     lenadd += len(variables[i])
